#Dimensionality Reduction: PCA, MDS

In [ ]:
# import some needed libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)
from pylab import *
rcParams['figure.figsize'] = (7, 7)
font = {'weight' : 'normal',
        'size'   : 14}
matplotlib.rc('font', **font)

In [ ]:
# this command makes sure we can plot inline in the notebook
%matplotlib inline

## Principal Component Analysis (PCA)

### PCA for heart disease classification

### The data

In [ ]:
# load the data from a text file
heart = pd.read_table('heart.txt', header=0)

#### Explore the data

#### Description of the data
0. age: age in years
- sex: sex
    + 1 = male
    + 0 = female
- cp: chest pain type
    + 1 = typical angina
    + 2 = atypical angina
    + 3 = non-anginal pain
    + 4 = asymptomatic
- trestbps: resting blood pressure in mm Hg
- chol: serum cholestoral in mg/dl
- fbs: fasting blood sugar > 120 mg/dl
    + 1 = true
    + 0 = false
- restecg: resting electrocardiographic results
    + 0 = normal
    + 1 = having ST-T wave abnormality
    + 2 = showing probable or definite left ventricular hypertrophy
- thalach: maximum heart rate achieved
- exang: exercise induced angina
    + 1 = yes
    + 0 = no
- oldpeak: ST depression induced by exercise relative to rest
- slope: the slope of the peak exercise ST segment
    + 1 = upsloping
    + 2 = flat
    + 3 = downsloping
- ca: number of major vessels (0-3) colored by flourosopy
- thal: thallium scan -- the myocardial perfusion pattern 
    + 3 = normal
    + 6 = fixed defect
    + 7 = reversable defect
- target: diagnosis of heart disease
    + 0 = healthy subject
    + 1 = ill subject

- **Exercise 1**: Explore the data: print the number of samples, the number of features. How many samples does the dataset contain? How many features?    
<p>&nbsp; </p>
                    Which variables are categorical? Which ones are continuous?
<p>&nbsp; </p>
<p>&nbsp; </p>
<p>&nbsp; </p>

- **Exercise 2**: How many healthy subjects are in the dataset? How many ill subjects? (tip: call the *pd.value_counts* function on the *heart.target* column).
<p>&nbsp; </p>

#### Visualize the data

In [ ]:
# we need some colors for our plots!
colors = ['red', 'green', 'blue', 'yellow', 'cyan', 'pink', 'orange', 'purple']
from itertools import cycle

Our dataset contains both *continuos* and *categorical* variables.

Because we will visualize them in a different way, let's split the dataset like this:

In [ ]:
# the continuous variables
continuous = heart[['age','trestbps', 'chol', 'thalach', 'oldpeak']]
# the categorical variables
categorical = heart[['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']]

Let's start with a boxplot of our _continuous_ variables:

In [ ]:
bp = continuous.boxplot(patch_artist=True)
for patch, color in zip(bp['boxes'], cycle(colors)):
    patch.set_facecolor(color)
    patch.set_alpha(.6)
plt.title('Boxplot continuous variables')

- **Exercise 3**: What can you say about the distribution of these features? Do you think any sort of data pre-processing is required before performing a PCA?
<p>&nbsp; </p>
<p>&nbsp; </p>

And now some barplots for our _categorical_ variables:

In [ ]:
fig = plt.figure(figsize=(11,7))
fig.suptitle('Categorical variables', fontsize=14)
# loop for the plotting
for i in range(len((categorical.columns))):
    vec = categorical[categorical.columns[i]]
    ax = plt.subplot(2, 4, i+1)
    vec.value_counts().plot(ax=ax, kind='bar', color=colors[i], title=categorical.columns[i])

- **Exercise 4**: Do you see anything strange in the values of the two variables _major vessels_ and _thallium scan_? What?
<p>&nbsp; </p>

#### Dealing with missing data

In [ ]:
# let's convert the dataframe to a numpy array (just for convenience!)
heart_data = heart.values
i = np.where(heart_data=='?')[0]
print 'The dataset contains %i missing values' %len(i)

In [ ]:
# a list of True and False to flag if a value is missing or not
missing = np.in1d(range(heart_data.shape[0]), i)
# now we remove the missing values
heart_data_clean = heart_data[~missing]

- **Exercise 5**: How many samples does the dataset contain after missing data removal?
<p>&nbsp; </p>

In [ ]:
# let's now skip the last column (which is the target, and not a feature)
X = heart_data_clean[:, 0:heart_data_clean.shape[1]-1] 
X = np.array(X, dtype=float)
# our y (target) vector is the last column
y = np.asarray(heart_data_clean[:, heart_data_clean.shape[1]-1], dtype=int)

### Principal Component Analysis

Let's run a PCA on our data set with the **sklearn** package:

In [ ]:
# import the library
from sklearn.decomposition import PCA

In [ ]:
# create a model and fit and transform the data
pca = PCA()
X_pca = pca.fit_transform(X)

#### Inspecting the PCA results

Let's have a look at the the proportion of variance explained by each component:

In [ ]:
print pca.explained_variance_ratio_

- **Exercise 6**: Which is the percentage of variance explained by the first component? And the one explained by the second?
<p>&nbsp; </p>
<p>&nbsp; </p>

In order to decide how many principal components you should retain, you can summarise the results of the performed PCA by making a **scree plot**.  A scree plot shows the fraction of total variance in the data as explained or represented by each PC.

In [ ]:
# render the screeplot
plt.plot(pca.explained_variance_, 'ro-', linewidth=2)
plt.xlabel('n_components')
plt.ylabel('explained_variance_')
plt.title('PCA: screeplot', fontsize=14)

- **Exercise 7**: How many components do you think are needed to explain most of the variance of this dataset?
<p>&nbsp; </p>
<p>&nbsp; </p>

#### Visualize the data projection

In [ ]:
diagnose = y
diagnose_labels = ('healthy', 'ill')

In [ ]:
exercise = np.asarray(heart_data_clean[:, 8], dtype = int)
exercise_labels = ('exercise induced angina', 'non-exercise induced angina')

We define a function to quickly plot the data projection onto 2D:

In [ ]:
def plot2D(data, target, labels, atitle):
    fig, ax1 = plt.subplots(figsize=(7,7))
    plt.xlabel('coord 1')
    plt.ylabel('coord 2')
    for i, c in zip(np.unique(target), cycle(colors)):
        plt.scatter(data[target == i, 0], data[target == i, 1],
                  s=50, alpha=.6, c=c, label='%s - %s' %(i, labels[i]))
    plt.legend()
    plt.title(atitle)

In [ ]:
# let's plot the components highlighting healthy and ill subjects 
plot2D(X_pca, diagnose, diagnose_labels, 'PCA: heart disease dataset')

What you should have at this step is a projection of your 13 features samples on just 2 dimensions.
However, the two classes do not seem nicely separated one from the other. This happened because we did not standardize our data! 

So let's see what happens if we standardize the data set:

#### Standardize the data

In [ ]:
# import the library
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [ ]:
# we scale the dataset and store it as a dataframe
X_stand = StandardScaler().fit_transform(X)
stand = pd.DataFrame(X_stand, columns=heart.columns[0: len(heart.columns)-1])

- **Exercise 8**: Using some code from above (copy and paste!) make a boxplot of the continuous variables. How does the standardization affect the features?
<p>&nbsp; </p>
<p>&nbsp; </p>

Let's now run another PCA on our standardized data this time:

In [ ]:
X_pca_stand = pca.fit_transform(X_stand)

- **Exercise 9**: Make a screeplot for the PCA just obtained.
<p>&nbsp; </p>
<p>&nbsp; </p>

And the new data projection is:

In [ ]:
plot2D(X_pca_stand, diagnose, diagnose_labels, 'PCA: heart disease standardized dataset')

The two classes are now more easily separable from each other.

Now we take only the components associated with ill subjects (where diagnose==1), and we plot the 2D projection accordingly:


In [ ]:
pca_ill = X_pca_stand[diagnose==1,:]
# and these are the 'exercise' values for the ill subjects
exercise_ill = exercise[diagnose==1]

In [ ]:
# now we plot the components for the ill subjects, highlighting if they had an exercise-induced angina or not
plot2D(pca_ill, exercise_ill, exercise_labels, 'PCA: exercise induced angina?')

- **Exercise 10**: Do you think is it reasonably easy to classify ill subjects in those who had an exercise-induced angina and those who did not?
<p>&nbsp; </p>
<p>&nbsp; </p>

# Multi Dimensional Scaling (MDS)

## MDS for heart disease classification

We will first use MDS to visualize the heart disease dataset we just analyzed.

In [ ]:
# import the library we will need
from sklearn import manifold

In [ ]:
mds = manifold.MDS(n_components=2)

In [ ]:
# run the MDS
Y_mds = mds.fit_transform(X_stand)

In [ ]:
# plot the 2D projection (we can simply reuse the function we defined before!)
plot2D(Y_mds, diagnose, diagnose_labels, 'MDS: heart disease dataset')

- **Exercise 11**: This 2D projection is somehow different from the one obtained through a PCA. Can you explain how the two techniques are different?
<p>&nbsp; </p>
<p>&nbsp; </p>

## MDS for biological response prediction

We will now use MDS for a different dataset.

### The data

In [ ]:
# load the data from a text file
response = pd.read_csv('biol_response.csv', header=0)

- **Exercise 12**: Have a look at the data. How many samples does this dataset contain? How many features?
<p>&nbsp; </p>
<p>&nbsp; </p>

###**Description of the biological response dataset**
This dataset has been used in a _kaggle_ competition to predict biological response of molecules from their chemical properties.
Each row in this data set represents a molecule. The first column contains experimental data describing an actual biological response; the molecule was seen to elicit this response (1), or not (0). The remaining columns represent molecular descriptors (d1 through d1776), these are calculated properties that can capture some of the characteristics of the molecule - for example size, shape, or elemental constitution. The descriptor matrix has been normalized. For more information on the challenge: https://www.kaggle.com/c/bioresponse.

- **Exercise 13**: Get the unique counts of the _Activity_ variable (again use some code from above!). How many active samples does the dataset contain? How many inactive samples?
<p>&nbsp; </p>

#### Visualize the data

This is a big dataset, let's just select 15 variables and plot them:

In [ ]:
columns = np.random.choice(response.columns.values, 50)

In [ ]:
plt.figure(figsize=(15,7))
bp = response[columns].boxplot(patch_artist=True)
plt.xticks(rotation=90) # set the label on the x axis rotated in vertical
for patch, color in zip(bp['boxes'], cycle(colors)):
    patch.set_facecolor(color)
    patch.set_alpha(.6)
plt.title('Boxplot 50 random variables', fontsize=14)

- **Exercise 14**: This dataset is already normalized. Can you explain in which way? Which normalization method has been applied to the data?
<p>&nbsp; </p>
<p>&nbsp; </p>
<p>&nbsp; </p>

In [ ]:
# we now convert the dataframe to a numpy array
X = response.values
X = X [:, 1:X.shape[1]] #skip first column
activity = np.asarray(response.Activity) # numeric target (y)
activity_labels = ('inactive', 'active')

Now we run the MDS: this is a big dataset, it will take some time...

In [ ]:
Y_mds = mds.fit_transform(X)

We can now just use the same function of before and plot the 2D projection:

In [ ]:
plot2D(Y_mds, activity, activity_labels, 'MDS: biological response dataset')

This doesn't look too nice, does it? There are clearly some clusters, but the overlapping between active and inactive samples is too big to be able to actually tell the two classes apart. The MDS is clearly not the most appropriate technique to visualize this dataset.

## MDS for Yeast Gene Expression Classification

Let's see if the MDS does a better job on Yeast gene expression data.

In [ ]:
# import the file from a text file
yeast = pd.read_table('yeast.csv', sep = ' ', header=0)

- **Exercise 15**: Have a look at the data. How many samples does this dataset contain? How many features?
<p>&nbsp; </p>
<p>&nbsp; </p>

#### Visualize the data

In [ ]:
plt.figure(figsize=(15,7))
cols = [col for col in yeast.columns if col not in ['gene_class']]
bp = yeast[cols].boxplot(patch_artist=True)
plt.xticks(rotation=90)
for patch, color in zip(bp['boxes'], cycle(colors)):
    patch.set_facecolor(color)
    patch.set_alpha(.6)
plt.title('Boxplot: yeast gene expressions', fontsize=14)

- **Exercise 16**: Do the features have all the same range? If yes, which one?
<p>&nbsp; </p>

- **Exercise 17**: Get the unique counts of the Label variable. How many classes are there for the genes? Which is the most abundant class?
<p>&nbsp; </p>
<p>&nbsp; </p>

In [ ]:
# let's get rid of some columns we do not need for the MDS
cols = [col for col in yeast.columns if col not in ['ORF', 'Label','gene_class']]
X = np.asarray(yeast[cols])

In [ ]:
# run the MDS
Y_mds = mds.fit_transform(X)

In [ ]:
gene_class = np.asarray(yeast.gene_class)
gene_labels = pd.unique(yeast.Label)

In [ ]:
# plot the 2D projection
plot2D(Y_mds, gene_class, gene_labels, 'MDS: yeast gene expression dataset')

Let's just focus on _Ribo_ and _nc_ (non-classified) genes, and look at the 2D projection.

In [ ]:
subset_mds = Y_mds[(gene_class==2)|(gene_class==5),:]
subset_gene_class = gene_class[(gene_class==2)|(gene_class==5)]
plot2D(subset_mds, subset_gene_class, gene_labels, 'MDS: Ribo genes')

- **Exercise 18**: Can you easily distinguish the Ribo genes from the rest?
<p>&nbsp; </p>

- **Exercise 19**: By default, the __Euclidean__ distance is used in the MDS to compute the dissimilarities between the data points. Do you know any other metrics that could be used instead to represent similarities/dissimilarities?
<p>&nbsp; </p>
<p>&nbsp; </p>